In [38]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


In [2]:
import utils

In [3]:
itp_id = 300

In [4]:
pd.set_option("display.max_columns", 100)


In [5]:
analysis_date_start = '2022-05-01'
analysis_date_end = '2022-05-31'

In [6]:
gtfs_daily = utils.load_schedule_data(analysis_date_start, analysis_date_end, itp_id)

In [7]:
gtfs_daily

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,6242274318901953391,882298,3487,300,0,12010,2022-05-13,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,12,12,51240,53340,0.583333
1,4100255323842349957,6242274318901953391,882298,3487,300,0,12010,2022-05-03,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,12,12,51240,53340,0.583333
2,4100255323842349957,-7852302470817784509,882500,3489,300,0,10,2022-05-16,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14,14,36660,38040,0.383333
3,4100255323842349957,1786367904991358065,882503,3489,300,0,10,2022-05-10,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14,14,39540,40920,0.383333
4,4100255323842349957,-4424835700699550644,882531,3489,300,0,10,2022-05-05,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14,14,59700,61080,0.383333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39725,4100255323842349957,1685620346522458701,881934,3485,300,0,10,2022-05-06,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,81300,84000,0.750000
39726,4100255323842349957,-4850510893495675724,881956,3485,300,0,10,2022-05-03,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,23520,26880,0.933333
39727,4100255323842349957,-7046476738018100689,881778,3485,300,0,2,2022-05-08,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,48420,52800,1.216667
39728,4100255323842349957,6797982000711812649,881649,3485,300,0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,50040,54360,1.200000


In [8]:
gtfs_daily.service_date.min()

datetime.date(2022, 5, 1)

In [9]:
gtfs_daily.service_date.max()

datetime.date(2022, 5, 31)

In [10]:
rt = utils.load_rt_data(analysis_date_start, analysis_date_end)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [11]:
rt

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id
0,300,0,2022-05-01,882742,1301
1,300,0,2022-05-01,882736,1301
2,300,0,2022-05-01,882739,1301
3,300,0,2022-05-01,882771,1301
4,300,0,2022-05-01,882761,1301
...,...,...,...,...,...
38896,300,0,2022-05-31,881885,5316
38897,300,0,2022-05-31,883406,5316
38898,300,0,2022-05-31,883414,5316
38899,300,0,2022-05-31,883385,5316


In [12]:
rt['str_len'] = rt.trip_id.str.len()

In [13]:
rt.str_len.value_counts()

6    38901
Name: str_len, dtype: int64

In [14]:
len(rt)

38901

In [15]:
rt.trip_id.nunique()

3054

In [16]:
len(gtfs_daily)

39730

In [17]:
gtfs_daily.trip_id.nunique()

3158

In [18]:
date_list =['2022-05-01','2022-05-02', '2022-05-31']

In [19]:

from datetime import date, timedelta
start_date = date(2022, 5, 1)
end_date = date(2022, 5, 31)

In [20]:
# daterange = pd.date_range(start_date, end_date)

In [21]:
# (daterange)

In [22]:
#above range does not work with function

In [23]:
dates = set()

def daterange2(start, end):
    for n in range(int((end - start).days) + 1):
        yield start + timedelta(n)

for single_date in daterange2(start_date, end_date):
    dates.add(single_date.strftime('%Y-%m-%d'))

In [24]:
(dates)

{'2022-05-01',
 '2022-05-02',
 '2022-05-03',
 '2022-05-04',
 '2022-05-05',
 '2022-05-06',
 '2022-05-07',
 '2022-05-08',
 '2022-05-09',
 '2022-05-10',
 '2022-05-11',
 '2022-05-12',
 '2022-05-13',
 '2022-05-14',
 '2022-05-15',
 '2022-05-16',
 '2022-05-17',
 '2022-05-18',
 '2022-05-19',
 '2022-05-20',
 '2022-05-21',
 '2022-05-22',
 '2022-05-23',
 '2022-05-24',
 '2022-05-25',
 '2022-05-26',
 '2022-05-27',
 '2022-05-28',
 '2022-05-29',
 '2022-05-30',
 '2022-05-31'}

In [25]:
rt['date'] = pd.to_datetime(rt['date'])

In [26]:
gtfs_daily['service_date'] = pd.to_datetime(gtfs_daily['service_date'])

In [27]:
single_date2 = '2022-05-08'

(rt>>filter(_.date == single_date2))


,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id,str_len
8910,300,0,2022-05-08,882227,1303,6
8911,300,0,2022-05-08,882219,1303,6
8912,300,0,2022-05-08,882246,1303,6
8913,300,0,2022-05-08,882212,1303,6
8914,300,0,2022-05-08,882217,1303,6
...,...,...,...,...,...,...
9692,300,0,2022-05-08,881140,5317,6
9693,300,0,2022-05-08,881211,5317,6
9694,300,0,2022-05-08,881201,5317,6
9695,300,0,2022-05-08,881356,5317,6


In [28]:
# def get_pct_ran_df2(itp_id, list_of_dates):
#     pcts = []
#     for single_date in list_of_dates:
#         gtfs_daily2 = (gtfs_daily>>filter(_.service_date == single_date))
#         rt2 = (rt>>filter(_.date == single_date))
#         sched_rt_df = (pd.merge(gtfs_daily2, rt2, how='outer', on='trip_id', indicator='have_rt'))
#         #pct_ran = (utils.get_pct_ran(sched_rt_df, single_date))
#         day_pct_ran = {}
#         day_pct_ran['date'] = single_date
#         if ((len(sched_rt_df))!=0):
#             day_pct_ran['pct_trips_ran'] = ((len(sched_rt_df>>filter(_.have_rt=='both')))/(len(gtfs_daily2)))
#         elif ((len(sched_rt_df))==0):
#             day_pct_ran['pct_trips_ran'] = ''
#         pct_ran = pd.DataFrame([day_pct_ran])
#         pct_ran['n_have_rt'] = (len(sched_rt_df>>filter(_.have_rt=='both')))
#         pct_ran['n_missing_rt'] = (len(sched_rt_df>>filter(_.have_rt=='right_only')))
#         pct_ran['unmatched_rt'] = (len(sched_rt_df>>filter(_.have_rt=='left_only')))
#         pct_ran['nunique_sched'] = (gtfs_daily2.trip_id.nunique())
#         pct_ran['nunique_rt'] = (rt2.trip_id.nunique())

#         pcts.append(pct_ran)                                                    
#         #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
#     pcts = pd.concat(pcts)
#     pcts = pcts>>arrange(_.date)
#     return pd.DataFrame(pcts)


In [29]:
may = utils.get_pct_ran_df(itp_id, dates, gtfs_daily, rt)

In [30]:
may

,date,pct_trips_ran,n_have_rt,n_missing_rt,unmatched_rt,nunique_sched,nunique_rt
0,2022-05-01,0.684416,527,252,244,770,778
0,2022-05-02,0.705145,1069,252,450,1516,1318
0,2022-05-03,0.971636,1473,0,151,1516,1365
0,2022-05-04,0.980871,1487,0,135,1516,1381
0,2022-05-05,0.959763,1455,0,173,1516,1343
0,2022-05-06,0.950820,1392,0,169,1464,1295
0,2022-05-07,0.698851,608,395,266,870,999
0,2022-05-08,0.654545,504,283,269,770,783
0,2022-05-09,0.699208,1060,247,463,1516,1300
0,2022-05-10,0.977573,1482,0,136,1516,1380


In [31]:
import altair as alt
from dla_utils import _dla_utils

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [32]:
bar = (alt.Chart(may)
        .mark_bar()
        .encode(
            x=alt.X('date', title=_dla_utils.labeling('date'), sort=("x")),
            y=alt.Y('pct_trips_ran', title=_dla_utils.labeling('pct_trips_ran')),
            color=alt.Color(
                'pct_trips_ran',
                scale=alt.Scale(range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                legend=alt.Legend(title=(_dla_utils.labeling('pct_trips_ran')), symbolLimit=10)
            )
        ).properties(title='"Percent of Scheduled Trips with RT Data"'))
    
chart = styleguide.preset_chart_config(bar)
chart

alt.Chart(...)

In [33]:
may.sample()

,date,pct_trips_ran,n_have_rt,n_missing_rt,unmatched_rt,nunique_sched,nunique_rt
0,2022-05-16,0.68,1030,255,488,1516,1281


In [34]:
may_long =  (may>>select(_.date,
                         _.nunique_sched,
                         _.nunique_rt, 
             #            _.n_have_rt
                        ) 
             >> gather('measure',
                       'value',
                       _.nunique_sched,
                       _.nunique_rt,
       #                _.n_have_rt
                      )
            )

In [35]:
may_long

,date,measure,value
0,2022-05-01,nunique_sched,770
1,2022-05-02,nunique_sched,1516
2,2022-05-03,nunique_sched,1516
3,2022-05-04,nunique_sched,1516
4,2022-05-05,nunique_sched,1516
...,...,...,...
57,2022-05-27,nunique_rt,1320
58,2022-05-28,nunique_rt,1002
59,2022-05-29,nunique_rt,814
60,2022-05-30,nunique_rt,741


In [36]:
bar = alt.Chart(may_long).mark_bar(opacity=0.7).encode(
    x='date:O',
    y=alt.Y('value:Q', stack=None),
    color="measure",
)

chart = styleguide.preset_chart_config(bar)
chart

alt.Chart(...)

In [37]:
base = alt.Chart(may).encode(x=alt.X('date:O', title=_dla_utils.labeling('date')))

bar = base.mark_bar().encode(y=alt.Y('nunique_sched:Q', title=_dla_utils.labeling('nunique_sched')))

line =  base.mark_line(color='#EB9F3C').encode(
    y='nunique_rt:Q'
)

(bar + line).properties(title="Number of Unique Scheduled Trips and Number of Unique RT Trips")

alt.LayerChart(...)